In [1]:
import os
os.chdir(os.path.join(os.getcwd(), '..','..', '..'))

In [2]:
import numpy as np
import csv
import pandas as pd
import collections
from dataset.preprocessing.tf_idf_all_feature_matrix_gen import TFIDFFeatureGeneration
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import f1_score
from sklearn.model_selection import KFold
from sklearn.feature_selection import SelectKBest, chi2
from sklearn.ensemble import ExtraTreesClassifier
import weka.core.jvm as jvm
from weka.core.converters import Loader
from weka.filters import Filter
from weka.attribute_selection import ASEvaluation, AttributeSelection
from weka.classifiers import Classifier, Evaluation
from imblearn.over_sampling import SMOTE

jvm.start()

In [3]:
class DecisionTree:
    def __init__(self, x_train, y_train, x_test, y_test, k):
        self.dtc = DecisionTreeClassifier(splitter='random', random_state=42)
        self.x_train = x_train
        self.y_train = y_train
        self.x_test = x_test
        self.y_test = y_test
        self.k = k
        # print(self.x_train.shape, self.y_train.shape, self.x_test.shape, self.y_test.shape)
        
    def feature_selection_SelectKBest(self):
        k_best = SelectKBest(chi2, k=self.k)
        k_best.fit(self.x_train, self.y_train)
        self.x_train = k_best.transform(self.x_train)
        self.x_test = k_best.transform(self.x_test)
        
    def feature_selection_ExtraTreesClassifier(self):
        clf = ExtraTreesClassifier(n_estimators=100, random_state=42)
        clf.fit(self.x_train, self.y_train)
        importances = clf.feature_importances_
        indices = np.argsort(importances)[::-1]
        self.x_train = self.x_train[:, indices[:self.k]]
        self.x_test = self.x_test[:, indices[:self.k]]

    def feature_selection_InfoGainAttributeEval(self, morbidity):
        loader = Loader(classname="weka.core.converters.ArffLoader")
        train_data = loader.load_file(f"./dataset/train/train_{morbidity}_tfidf.arff")
        train_data.class_is_last()

        # Initialize attribute selection
        eval = ASEvaluation(classname="weka.attributeSelection.InfoGainAttributeEval")
        search = AttributeSelection()
        search.evaluator = eval
        search.select_attributes(train_data)
        selected_attributes = search.selected_attributes
        filtered_attributes = np.delete(selected_attributes, [-1])
        # print("Selected attributes:", type(filtered_attributes), filtered_attributes.shape)

        # Apply selected attributes to the training and testing sets
        self.x_train = self.x_train[:, filtered_attributes]
        self.x_test = self.x_test[:, filtered_attributes]

    def train(self):
        self.dtc.fit(self.x_train, self.y_train)

    def test_and_evaluate(self):
        y_pred = self.dtc.predict(self.x_test)
        f1_macro = f1_score(self.y_test, y_pred, average='macro')
        f1_micro = f1_score(self.y_test, y_pred, average='micro')
        #print(f"Macro F1 score: {f1_macro} and Micro F1 Score {f1_micro}")
        return f1_macro, f1_micro

In [4]:
morbidities = ['Asthma', 'CAD', 'CHF', 'Depression', 'Diabetes', 'Gallstones', 'GERD', 'Gout', 'Hypercholesterolemia', 'Hypertension', 'Hypertriglyceridemia', 'OA', 'Obesity', 'OSA', 'PVD', 'Venous-Insufficiency']

column_headings = ["Morbidity Class", "DT_Macro F1", "DT_Micro F1"]
    

### All Features

In [5]:
with open("./results/tf-idf-features/performance_DT_AllFeatures.csv", "w", newline="") as file:
    writer = csv.writer(file)
    writer.writerow([column_headings[0], column_headings[1], column_headings[2]])
    
all_f1_macro_scores = []
all_f1_micro_scores = []

for morbidity in morbidities:
    print(morbidity)
    train_preprocessed_df = pd.read_csv('./dataset/train/train_data_intuitive_preprocessed.csv')
    train_preprocessed_df = train_preprocessed_df[train_preprocessed_df[morbidity].isin([1.0, 0.0])]

    X, Y, words = TFIDFFeatureGeneration(train_preprocessed_df, morbidity).tf_idf_matrix_gen()

    if len(collections.Counter(list(Y)).keys()) >=2:
        print(morbidity)
        smote = SMOTE(random_state=42,k_neighbors=2)
        X_train_resampled, y_train_resampled = smote.fit_resample(X, Y)
        X, Y =  X_train_resampled, y_train_resampled

        # add KFold cross validation
        skf = KFold(n_splits=10, shuffle=True, random_state=42)

        f1_macro_list = []
        f1_micro_list = []
        for train_idx, val_idx in skf.split(X, Y):
            X_train_fold, Y_train_fold = X[train_idx], Y[train_idx]
            X_val_fold, Y_val_fold = X[val_idx], Y[val_idx]

            # Training RF using TF-IDF Representation
            dt_obj = DecisionTree(X_train_fold, Y_train_fold, X_val_fold, Y_val_fold, 0)
            dt_obj.train()

            f1_macro, f1_micro = dt_obj.test_and_evaluate()

            f1_macro_list.append(f1_macro)
            f1_micro_list.append(f1_micro)

        f1_macro = np.mean(f1_macro_list)
        f1_micro = np.mean(f1_micro_list)
    else:
        f1_macro = 1
        f1_micro = 1
    
    print(f"Macro F1 score: {f1_macro} and Micro F1 Score {f1_micro}")

    row_heading = morbidity

    # data to be written to the CSV file
    data = [f1_macro, f1_micro]
    all_f1_macro_scores.append(f1_macro)
    all_f1_micro_scores.append(f1_micro)

    with open("./results/tf-idf-features/performance_DT_AllFeatures.csv", "a", newline="") as file:
        writer = csv.writer(file)
        row = [row_heading]
        row.extend(data)
        writer.writerow(row)


with open("./results/tf-idf-features/performance_DT_AllFeatures.csv", "a", newline="") as file:
    writer = csv.writer(file)
    row = ["Overall-Average"]
    row.extend([sum(all_f1_macro_scores)/len(all_f1_macro_scores),  sum(all_f1_micro_scores)/len(all_f1_micro_scores) ])
    writer.writerow(row)

Asthma
(572, 600) (572,) Counter({0.0: 502, 1.0: 70})
Asthma
Macro F1 score: 0.9726535664564213 and Micro F1 Score 0.9730891089108911
CAD
(548, 600) (548,) Counter({1.0: 325, 0.0: 223})
CAD
Macro F1 score: 0.8645333783737124 and Micro F1 Score 0.866153846153846
CHF
(243, 600) (243,) Counter({1.0: 243})
Macro F1 score: 1 and Micro F1 Score 1
Depression
(582, 600) (582,) Counter({0.0: 460, 1.0: 122})
Depression
Macro F1 score: 0.8842529934150823 and Micro F1 Score 0.8858695652173912
Diabetes
(567, 600) (567,) Counter({1.0: 396, 0.0: 171})
Diabetes
Macro F1 score: 0.9363384818600986 and Micro F1 Score 0.9369462025316455
Gallstones
(593, 600) (593,) Counter({0.0: 506, 1.0: 87})
Gallstones
Macro F1 score: 0.842897631894585 and Micro F1 Score 0.8448165404775771
GERD
(487, 600) (487,) Counter({0.0: 372, 1.0: 115})
GERD
Macro F1 score: 0.7826061105055026 and Micro F1 Score 0.784918918918919
Gout
(596, 600) (596,) Counter({0.0: 518, 1.0: 78})
Gout
Macro F1 score: 0.8641346787007891 and Micro F1

### Select K Best k=100

In [6]:
with open("./results/tf-idf-features/performance_DT_SelectKBest.csv", "w", newline="") as file:
    writer = csv.writer(file)
    writer.writerow([column_headings[0], column_headings[1], column_headings[2]])
    
all_f1_macro_scores = []
all_f1_micro_scores = []

for morbidity in morbidities:
    print(morbidity)
    train_preprocessed_df = pd.read_csv('./dataset/train/train_data_intuitive_preprocessed.csv')
    train_preprocessed_df = train_preprocessed_df[train_preprocessed_df[morbidity].isin([1.0, 0.0])]

    X, Y, words = TFIDFFeatureGeneration(train_preprocessed_df, morbidity).tf_idf_matrix_gen()

    if len(collections.Counter(list(Y)).keys()) >=2:
        print(morbidity)
        smote = SMOTE(random_state=42,k_neighbors=2)
        X_train_resampled, y_train_resampled = smote.fit_resample(X, Y)
        X, Y =  X_train_resampled, y_train_resampled
        
        # add KFold cross validation
        skf = KFold(n_splits=10, shuffle=True, random_state=42)

        f1_macro_list = []
        f1_micro_list = []
        for train_idx, val_idx in skf.split(X, Y):
            X_train_fold, Y_train_fold = X[train_idx], Y[train_idx]
            X_val_fold, Y_val_fold = X[val_idx], Y[val_idx]

            # Training RF using TF-IDF Representation
            dt_obj = DecisionTree(X_train_fold, Y_train_fold, X_val_fold, Y_val_fold, 100)
            dt_obj.feature_selection_SelectKBest()
            dt_obj.train()   

            f1_macro, f1_micro = dt_obj.test_and_evaluate()

            f1_macro_list.append(f1_macro)
            f1_micro_list.append(f1_micro)

        f1_macro = np.mean(f1_macro_list)
        f1_micro = np.mean(f1_micro_list)
    else:
        f1_macro = 1
        f1_micro = 1
    
    print(f"Macro F1 score: {f1_macro} and Micro F1 Score {f1_micro}")

    row_heading = morbidity

    # data to be written to the CSV file
    data = [f1_macro, f1_micro]
    all_f1_macro_scores.append(f1_macro)
    all_f1_micro_scores.append(f1_micro)

    with open("./results/tf-idf-features/performance_DT_SelectKBest.csv", "a", newline="") as file:
        writer = csv.writer(file)
        row = [row_heading]
        row.extend(data)
        writer.writerow(row)


with open("./results/tf-idf-features/performance_DT_SelectKBest.csv", "a", newline="") as file:
    writer = csv.writer(file)
    row = ["Overall-Average"]
    row.extend([sum(all_f1_macro_scores)/len(all_f1_macro_scores),  sum(all_f1_micro_scores)/len(all_f1_micro_scores) ])
    writer.writerow(row)

Asthma
(572, 600) (572,) Counter({0.0: 502, 1.0: 70})
Asthma
Macro F1 score: 0.9788647566848031 and Micro F1 Score 0.979069306930693
CAD
(548, 600) (548,) Counter({1.0: 325, 0.0: 223})
CAD
Macro F1 score: 0.8787201632148964 and Micro F1 Score 0.8800000000000001
CHF
(243, 600) (243,) Counter({1.0: 243})
Macro F1 score: 1 and Micro F1 Score 1
Depression
(582, 600) (582,) Counter({0.0: 460, 1.0: 122})
Depression
Macro F1 score: 0.8570808539655891 and Micro F1 Score 0.8586956521739131
Diabetes
(567, 600) (567,) Counter({1.0: 396, 0.0: 171})
Diabetes
Macro F1 score: 0.9389233397420182 and Micro F1 Score 0.9393512658227847
Gallstones
(593, 600) (593,) Counter({0.0: 506, 1.0: 87})
Gallstones
Macro F1 score: 0.8490187824752903 and Micro F1 Score 0.8517569403999223
GERD
(487, 600) (487,) Counter({0.0: 372, 1.0: 115})
GERD
Macro F1 score: 0.7520790312893025 and Micro F1 Score 0.7552792792792794
Gout
(596, 600) (596,) Counter({0.0: 518, 1.0: 78})
Gout
Macro F1 score: 0.8727867234076709 and Micro 

### ExtraTreesClassifier

In [7]:
with open("./results/tf-idf-features/performance_DT_ExtraTreesClassifier.csv", "w", newline="") as file:
    writer = csv.writer(file)
    writer.writerow([column_headings[0], column_headings[1], column_headings[2]])
    
all_f1_macro_scores = []
all_f1_micro_scores = []

for morbidity in morbidities:
    print(morbidity)
    train_preprocessed_df = pd.read_csv('./dataset/train/train_data_intuitive_preprocessed.csv')
    train_preprocessed_df = train_preprocessed_df[train_preprocessed_df[morbidity].isin([1.0, 0.0])]

    X, Y, words = TFIDFFeatureGeneration(train_preprocessed_df, morbidity).tf_idf_matrix_gen()

    if len(collections.Counter(list(Y)).keys()) >=2:
        print(morbidity)
        smote = SMOTE(random_state=42,k_neighbors=2)
        X_train_resampled, y_train_resampled = smote.fit_resample(X, Y)
        X, Y =  X_train_resampled, y_train_resampled
        
        # add KFold cross validation
        skf = KFold(n_splits=10, shuffle=True, random_state=42)

        f1_macro_list = []
        f1_micro_list = []
        for train_idx, val_idx in skf.split(X, Y):
            X_train_fold, Y_train_fold = X[train_idx], Y[train_idx]
            X_val_fold, Y_val_fold = X[val_idx], Y[val_idx]

            # Training RF using TF-IDF Representation
            dt_obj = DecisionTree(X_train_fold, Y_train_fold, X_val_fold, Y_val_fold, 100)
            dt_obj.feature_selection_ExtraTreesClassifier()
            dt_obj.train()

            f1_macro, f1_micro = dt_obj.test_and_evaluate()

            f1_macro_list.append(f1_macro)
            f1_micro_list.append(f1_micro)

        f1_macro = np.mean(f1_macro_list)
        f1_micro = np.mean(f1_micro_list)
    else:
        f1_macro = 1
        f1_micro = 1
    
    print(f"Macro F1 score: {f1_macro} and Micro F1 Score {f1_micro}")

    row_heading = morbidity

    # data to be written to the CSV file
    data = [f1_macro, f1_micro]
    all_f1_macro_scores.append(f1_macro)
    all_f1_micro_scores.append(f1_micro)

    with open("./results/tf-idf-features/performance_DT_ExtraTreesClassifier.csv", "a", newline="") as file:
        writer = csv.writer(file)
        row = [row_heading]
        row.extend(data)
        writer.writerow(row)


with open("./results/tf-idf-features/performance_DT_ExtraTreesClassifier.csv", "a", newline="") as file:
    writer = csv.writer(file)
    row = ["Overall-Average"]
    row.extend([sum(all_f1_macro_scores)/len(all_f1_macro_scores),  sum(all_f1_micro_scores)/len(all_f1_micro_scores) ])
    writer.writerow(row)

Asthma
(572, 600) (572,) Counter({0.0: 502, 1.0: 70})
Asthma
Macro F1 score: 0.9759248241674555 and Micro F1 Score 0.9760792079207923
CAD
(548, 600) (548,) Counter({1.0: 325, 0.0: 223})
CAD
Macro F1 score: 0.8571081967693465 and Micro F1 Score 0.8584615384615386
CHF
(243, 600) (243,) Counter({1.0: 243})
Macro F1 score: 1 and Micro F1 Score 1
Depression
(582, 600) (582,) Counter({0.0: 460, 1.0: 122})
Depression
Macro F1 score: 0.8511979204266537 and Micro F1 Score 0.8543478260869565
Diabetes
(567, 600) (567,) Counter({1.0: 396, 0.0: 171})
Diabetes
Macro F1 score: 0.9453028934295897 and Micro F1 Score 0.9457278481012658
Gallstones
(593, 600) (593,) Counter({0.0: 506, 1.0: 87})
Gallstones
Macro F1 score: 0.8542502241509876 and Micro F1 Score 0.8566880217433509
GERD
(487, 600) (487,) Counter({0.0: 372, 1.0: 115})
GERD
Macro F1 score: 0.7784443604903009 and Micro F1 Score 0.780864864864865
Gout
(596, 600) (596,) Counter({0.0: 518, 1.0: 78})
Gout
Macro F1 score: 0.8675690034445008 and Micro 

### Info Gain

In [ ]:
with open("./results/tf-idf-features/performance_DT_InfoGain.csv", "w", newline="") as file:
    writer = csv.writer(file)
    writer.writerow([column_headings[0], column_headings[1], column_headings[2]])
    
all_f1_macro_scores = []
all_f1_micro_scores = []

for morbidity in morbidities:
    print(morbidity)
    train_preprocessed_df = pd.read_csv('./dataset/train/train_data_intuitive_preprocessed.csv')
    train_preprocessed_df = train_preprocessed_df[train_preprocessed_df[morbidity].isin([1.0, 0.0])]

    X, Y, words = TFIDFFeatureGeneration(train_preprocessed_df, morbidity).tf_idf_matrix_gen()

    
    if len(collections.Counter(list(Y)).keys()) < 2:
        f1_macro = 1
        f1_micro = 1
    else:
        print(morbidity)
        smote = SMOTE(random_state=42,k_neighbors=2)
        X_train_resampled, y_train_resampled = smote.fit_resample(X, Y)
        X, Y =  X_train_resampled, y_train_resampled
        
        # add KFold cross validation
        skf = KFold(n_splits=10, shuffle=True, random_state=42)

        f1_macro_list = []
        f1_micro_list = []
        for train_idx, val_idx in skf.split(X, Y):
            X_train_fold, Y_train_fold = X[train_idx], Y[train_idx]
            X_val_fold, Y_val_fold = X[val_idx], Y[val_idx]

            # Training RF using TF-IDF Representation
            dt_obj = DecisionTree(X_train_fold, Y_train_fold, X_val_fold, Y_val_fold, 100)
            dt_obj.feature_selection_InfoGainAttributeEval(morbidity)
            dt_obj.train()

            f1_macro, f1_micro = dt_obj.test_and_evaluate()

            f1_macro_list.append(f1_macro)
            f1_micro_list.append(f1_micro)

        f1_macro = np.mean(f1_macro_list)
        f1_micro = np.mean(f1_micro_list)
    print(f"Macro F1 score: {f1_macro} and Micro F1 Score {f1_micro}")

    row_heading = morbidity

    # data to be written to the CSV file
    data = [f1_macro, f1_micro]
    all_f1_macro_scores.append(f1_macro)
    all_f1_micro_scores.append(f1_micro)

    with open("./results/tf-idf-features/performance_DT_InfoGain.csv", "a", newline="") as file:
        writer = csv.writer(file)
        row = [row_heading]
        row.extend(data)
        writer.writerow(row)


with open("./results/tf-idf-features/performance_DT_InfoGain.csv", "a", newline="") as file:
    writer = csv.writer(file)
    row = ["Overall-Average"]
    row.extend([sum(all_f1_macro_scores)/len(all_f1_macro_scores),  sum(all_f1_micro_scores)/len(all_f1_micro_scores) ])
    writer.writerow(row)